In [ ]:
!pip install kaggle
!pip install tensorflow
!pip install matplotlib
!pip install scikit-learn


In [ ]:
# Kaggle veri setini indir
!kaggle datasets download -d riyaelizashaju/skin-disease-classification-image-dataset

# İndirilen veri setini çıkar
!unzip skin-disease-classification-image-dataset.zip -d skin_disease_dataset


Dataset URL: https://www.kaggle.com/datasets/riyaelizashaju/skin-disease-classification-image-dataset
License(s): unknown
skin-disease-classification-image-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  skin-disease-classification-image-dataset.zip
replace skin_disease_dataset/Split_smol/train/Actinic keratosis/ISIC_0024468.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import os

# Dosya yapısını kontrol et
for root, dirs, files in os.walk('skin_disease_dataset'):
    print(root)
    for name in dirs:
        print(os.path.join(root, name))
    for name in files:
        print(os.path.join(root, name))


skin_disease_dataset
skin_disease_dataset/Split_smol
skin_disease_dataset/Split_smol
skin_disease_dataset/Split_smol/val
skin_disease_dataset/Split_smol/train
skin_disease_dataset/Split_smol/val
skin_disease_dataset/Split_smol/val/Dermatofibroma
skin_disease_dataset/Split_smol/val/Melanoma
skin_disease_dataset/Split_smol/val/Actinic keratosis
skin_disease_dataset/Split_smol/val/Vascular lesion
skin_disease_dataset/Split_smol/val/Melanocytic nevus
skin_disease_dataset/Split_smol/val/Squamous cell carcinoma
skin_disease_dataset/Split_smol/val/Atopic Dermatitis
skin_disease_dataset/Split_smol/val/Tinea Ringworm Candidiasis
skin_disease_dataset/Split_smol/val/Benign keratosis
skin_disease_dataset/Split_smol/val/Dermatofibroma
skin_disease_dataset/Split_smol/val/Dermatofibroma/ISIC_0027216.jpg
skin_disease_dataset/Split_smol/val/Dermatofibroma/ISIC_0024973.jpg
skin_disease_dataset/Split_smol/val/Dermatofibroma/ISIC_0026313.jpg
skin_disease_dataset/Split_smol/val/Dermatofibroma/ISIC_0029824.

In [ ]:
train_dir = 'skin_disease_dataset/Skin_disease/train'
val_dir = 'skin_disease_dataset/Skin_disease/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')


FileNotFoundError: [Errno 2] No such file or directory: 'skin_disease_dataset/Skin_disease/train'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Veri dizinleri
base_dir = 'skin_disease_dataset/Skin_disease'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

# Veri artırma ve ölçekleme
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

# Eğitim verisi yükleyici
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

# Doğrulama verisi yükleyici
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')  # 10 sınıf
])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=25,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 32)


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(25)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
